---
Packages required
---

In [1]:
import pandas as pd
import numpy as np

import networkx as nx

from pyecharts.charts import Geo
from pyecharts import options as opts
from pyecharts.globals import ChartType, SymbolType

from typing import List, Set, Dict, Tuple, Union
from itertools import combinations
from collections import defaultdict
import networkx as nx

pd.set_option('display.max_rows',10)
pd.set_option('display.min_rows',10)

# Read Data

In [2]:
# Read the .dat file (assuming tab-separated values)
airports = pd.read_csv('../data/airports.dat', sep=',',header = None, encoding='utf-8')  # Change separator as needed
airports.columns = [
    "Airport ID", "Name", "City", "Country", 
    "IATA", "ICAO", "Latitude", "Longitude", 
    "Altitude", "Timezone", "DST", "Tz database time zone", 
    "Type", "Source"
]
routes = pd.read_csv('../data/routes.dat', sep=',',header = None, encoding='utf-8')  # Change separator as needed
routes.columns = [
    "Airline", "Airline ID", "Source airport", "Source airport ID", 
    "Destination airport", "Destination airport ID", "Codeshare", 
    "Stops", "Equipment"
]

In [3]:
airports.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [4]:
routes.head()

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [5]:
# check duplicated
airports.duplicated().sum(), routes.duplicated().sum()

(0, 0)

In [6]:
routes[routes['Destination airport'].str.len() > 3]

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment


In [7]:
routes[routes['Source airport'].str.len() > 3]

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment


All the aiports in routes use IATA

# Data Preprocess

## Columns Select

In [8]:
airports_s = airports.loc[:,['IATA','Latitude','Longitude']]
routes_s = routes.loc[:,['Source airport','Destination airport']]
# Delete all airports without IATA
airports_s = airports_s[airports_s['IATA']!='\\N']

In [9]:
# check null
airports_s.isnull().sum(), routes_s.isnull().sum()

(IATA         0
 Latitude     0
 Longitude    0
 dtype: int64,
 Source airport         0
 Destination airport    0
 dtype: int64)

----------
We select routes that had appeared n or more times, and select airports which is existed in the filtered route data.

## Route counts

In [10]:
#  Count the number of times each route appears
route_counts = routes_s.value_counts().reset_index()
route_counts.columns = ['Source airport', 'Destination airport', 'Count']

def filtered_routes_list(n,m=9999):
    # save as [('Source airport','Destination airport')]
    routes_list = [
    (row['Source airport'], row['Destination airport'])
    for _, row in route_counts[(route_counts['Count'] >= n) & (route_counts['Count'] < m)].iterrows()
]
    return routes_list

def filtered_routes_dict(n,m=9999):
    # save as {('Source airport','Destination airport'): count}
    routes_dict = {
    (row['Source airport'], row['Destination airport']): row['Count']
    for _, row in route_counts[(route_counts['Count'] >= n) & (route_counts['Count'] < m)].iterrows()
    }
    return routes_dict

routes_list = filtered_routes_list(5) # As the data is too big, we only choose routes that had appeared five or more times
routes_dict = filtered_routes_dict(5)
print("Length of routes list:", len(routes_list))

Length of routes list: 1621


## Flight counts in each airport (filtered by routes selected)

In [11]:
# Total takeoffs and landings at each airport
departure_counts = route_counts.groupby('Source airport')['Count'].sum().rename('Departures')
arrival_counts = route_counts.groupby('Destination airport')['Count'].sum().rename('Arrivals')
flight_counts = pd.concat([departure_counts, arrival_counts], axis=1).fillna(0)

def filtered_airports_counts_list(routes_list):
    # Save as [('Airport'),flight counts]
    
    # Extract all airports involved in the routes_list, and filter airports in flight_counts that are not included in routes_list
    filtered_flight_counts = flight_counts.loc[
        flight_counts.index.intersection({airport for route in routes_list for airport in route})
    ]
    # Convert to pycharts input format
    airports_counts_list = sorted(
        [
            (airport, int(departures + arrivals))
            for airport, departures, arrivals in filtered_flight_counts.itertuples(index=True, name=None)
        ],
        key=lambda x: x[1],  # sort by total flights counts
        reverse=True
    )
    return airports_counts_list

airports_counts_list = filtered_airports_counts_list(filtered_routes_list(5))
print("Length of flight counts list:", len(airports_counts_list))

Length of flight counts list: 385


## Add coordinate points in Geo()

In [12]:
for i in range(len(airports_s)):
    Geo().add_coordinate(
    name = airports_s.iloc[i,0],
    latitude = airports_s.iloc[i,1],
    longitude =  airports_s.iloc[i,2]
    )

----
Preliminary visualization
----

In [13]:
# Initialize the Geo object
c = Geo(init_opts=opts.InitOpts(width="1600px", height="800px")) # Full the screen
c.add_schema(maptype="world")

# Add airport scatter points
c.add(
    "Airports",
    data_pair=airports_counts_list,  # e.g., a list of tuples [("ATL", 100), ("JFK", 80), ...]
    type_=ChartType.EFFECT_SCATTER,
    color="red",
    symbol_size=3,
)

# Add airline routes for different ranges
n = 2
for i in range(n):
    color = f"rgb({255 - 250/n*(i+1)},160, {250/n*(i+1)})"
    c.add(
        f"Airline routes - counts [{5+i*2},{7+i*2})",
        data_pair=filtered_routes_list(5+i*2, 7+i*2),  # Replace with your route filtering function
        type_=ChartType.LINES,
        symbol_size=0,
        effect_opts=opts.EffectOpts(
            symbol=SymbolType.ARROW, symbol_size=3, color=color,  # Dynamic RGB color
        ),
        linestyle_opts=opts.LineStyleOpts(
            width=0.1, curve=0.4, opacity=0.4 + 0.6/n*i, type_="solid",color=color,  # Dynamic RGB color
        ),
    )
    
# Add the left airline routes
c.add(
    f"Airline routes - counts [{5+n*2},9999)",
    data_pair=filtered_routes_list(5+n*2),
    type_=ChartType.LINES,
    symbol_size = 0,
    effect_opts=opts.EffectOpts(        
        symbol=SymbolType.ARROW, symbol_size=3, color="blue"
    ),
    linestyle_opts=opts.LineStyleOpts(width = 0.1, curve=0.4, color = "blue",opacity = 1, type_ = 'solid'),
)

c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
c.set_global_opts(title_opts=opts.TitleOpts(title="Aiport and Routes (Basic)",pos_left = 'center'),                  
                  legend_opts = opts.LegendOpts(type_ = 'scroll',orient = 'vertical',page_button_position = 'start',
                                               pos_left = 'left'))
c.render('../echarts/RouteBased/Aiports&Routes.html') # save as .html
# c.render_notebook() # if you want to display the graph in notebook, add this line, and set (width="900px", height="500px")

'C:\\Users\\laoth\\Desktop\\study\\web\\echarts\\RouteBased\\Aiports&Routes.html'

# Community Detection

In [14]:
Community = nx.DiGraph()
weighted_edges = [(u, v, weight) for (u, v), weight in routes_dict.items()]
Community.add_weighted_edges_from(weighted_edges)

In [15]:
# add flight counts to each nodes
def create_nodes_count(nodes):
    nodes_count = {}
    for key in nodes.keys():
        nodes_count[key] = [
            (airport,count)
            for airport,count in airports_counts_list
            if airport in nodes[key]
        ]
    return nodes_count

## Clique detection

In [16]:
# Find all cliques in the graph
undirected_community = Community.to_undirected()
cliques = list(nx.find_cliques(undirected_community))
# cliques

### Clique data

In [17]:
# Create an empty dictionary to store the nodes by clique size
clique_nodes = {}

# Iterate through each detected clique
for clique in cliques:
    clique_length = len(clique)  # Get the size of the clique
    
    # If the clique length is already a key, add the nodes to the list of nodes for this length
    cli = clique.copy()
    if clique_length in clique_nodes:
        clique_nodes[clique_length] += cli
    else:
        clique_nodes[clique_length] = cli

# Remove duplicates by converting each list to a set and then back to a list
for length in clique_nodes:
    clique_nodes[length] = list(set(clique_nodes[length]))

clique_nodes = dict(sorted(clique_nodes.items()))
# clique_nodes

In [18]:
clique_nodes_count = create_nodes_count(clique_nodes)

In [19]:
# Assuming clique_length_nodes contains the cliques by length, and routes_list contains routes
clique_routes = {}

# Iterate through each route in routes_list
for route in routes_list:
    source_airport, destination_airport = route

    # Initialize variable to store the maximum clique length where both airports belong
    max_clique_length = -1

    # Check which clique length both airports belong to
    for clique_length, nodes in clique_nodes.items():
        if source_airport in nodes and destination_airport in nodes:
            max_clique_length = max(max_clique_length, clique_length)

    # If both airports are found in a clique, add the route to the corresponding clique length
    if max_clique_length != -1:
        if max_clique_length not in clique_routes:
            clique_routes[max_clique_length] = []
        clique_routes[max_clique_length].append(route)

clique_routes = dict(sorted(clique_routes.items()))
# clique_routes

### Draw Cliques

In [20]:
# Initialize the Geo object
c = Geo(init_opts=opts.InitOpts(width="1600px", height="800px")) # Full the screen
c.add_schema(maptype="world")

L_n = len(clique_nodes.keys())
L_e = len(clique_routes.keys())

# Add airport scatter points
for i, key in enumerate(clique_nodes.keys()):
    rate = i/L_n
    color = f"rgb({255 - 250*rate},0, 0)"
    c.add(
        f"Airports {key}-clique",
        data_pair=clique_nodes_count[key],
        type_=ChartType.EFFECT_SCATTER,
        color=color,
        symbol_size=3,
    )

for i, key in enumerate(clique_routes.keys()):
    # Add airline routes for different ranges
    rate = i/L_e
    color = f"rgb({255 - 250*rate},160, {250*rate})"
    c.add(
        f"Routes {key}-clique",
        data_pair=clique_routes[key],  # Replace with your route filtering function
        type_=ChartType.LINES,
        symbol_size=0,
        effect_opts=opts.EffectOpts(
            symbol=SymbolType.ARROW, symbol_size=3, color=color,  # Dynamic RGB color
        ),
        linestyle_opts=opts.LineStyleOpts(
            width=0.1, curve=0.4, opacity=0.4 + 0.6*rate, type_="solid",color=color,  # Dynamic RGB color
        ),
    )

c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
c.set_global_opts(title_opts=opts.TitleOpts(title="Aiport and Routes (Cliques)",pos_left = 'center'),
                  legend_opts = opts.LegendOpts(type_ = 'scroll',orient = 'vertical',page_button_position = 'start',
                                               pos_left = 'left'))
c.render('../echarts/RouteBased/cliques.html') # save as .html
# c.render_notebook() # if you want to display the graph in notebook, add this line, and set (width="900px", height="500px")

'C:\\Users\\laoth\\Desktop\\study\\web\\echarts\\RouteBased\\cliques.html'

## K-cores

In [21]:
def k_cores(G):
    k = 1
    k_cores_nodes = {}
    k_cores_edges = {}
    while True:
        # Compute k-core for the current value of k
        k_core = nx.k_core(G, k=k)
        
        # If no nodes are left, stop
        if not k_core.nodes:
            k -= 1
            break
        
        # Store nodes and edges of the current k-core
        k_cores_nodes[k] = list(k_core.nodes)
        k_cores_edges[k] = list(k_core.edges)
        
        #-------This section is to reduce the size of the image, but will be little different from the result of kcore
        # Subtract previous k-cores' nodes and edges
        if k > 1:
            k_cores_nodes[k-1] = list(set(k_cores_nodes[k-1]) - set(k_cores_nodes[k]))
            k_cores_edges[k-1] = list(set(k_cores_edges[k-1]) - set(k_cores_edges[k]))
            if k_cores_nodes[k-1] == [] and k_cores_edges[k-1] == []:
                del k_cores_nodes[k-1]
                del k_cores_edges[k-1]
        k += 1  # Increment k for the next iteration

    return k, k_cores_nodes, k_cores_edges

###  K-cores data

In [22]:
k, k_nodes, k_edges = k_cores(Community)
print('Max-k:',k)
# print("K-Cores (Nodes):", k_nodes)
# print("K-Cores (Edges):", k_edges)

Max-k: 18


In [23]:
k_nodes_count = create_nodes_count(k_nodes)

### Draw K-cores

In [24]:
# Initialize the Geo object
c = Geo(init_opts=opts.InitOpts(width="1600px", height="800px")) # Full the screen
c.add_schema(maptype="world")

L_n = len(k_nodes.keys())
L_e = len(k_edges.keys())

for i, key in enumerate(k_nodes.keys()):
    # Add airport scatter points
    rate = i/L_n
    color = f"rgb({255 - 250*rate},0, 0)"
    c.add(
        f"Airports {key}-core",
        data_pair=k_nodes_count[key],
        type_=ChartType.EFFECT_SCATTER,
        color=color,
        symbol_size=3,
    )

for i, key in enumerate(k_edges.keys()):
    # Add airline routes for different ranges
    rate = i/L_e
    color = f"rgb({255 - 250*rate},160, {250*rate})"
    c.add(
        f"Routes {key}-core",
        data_pair=k_edges[key],  # Replace with your route filtering function
        type_=ChartType.LINES,
        symbol_size=0,
        effect_opts=opts.EffectOpts(
            symbol=SymbolType.ARROW, symbol_size=3, color=color,  # Dynamic RGB color
        ),
        linestyle_opts=opts.LineStyleOpts(
            width=0.1, curve=0.4, opacity=0.4 + 0.6*rate, type_="solid",color=color,  # Dynamic RGB color
        ),
    )

c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
c.set_global_opts(title_opts=opts.TitleOpts(title="Aiport and Routes (K-cores)",pos_left = 'center'),
                  legend_opts = opts.LegendOpts(type_ = 'scroll',orient = 'vertical',page_button_position = 'start',
                                               pos_left = 'left'))
c.render('../echarts/RouteBased/k-cores.html') # save as .html
# c.render_notebook() # if you want to display the graph in notebook, add this line, and set (width="900px", height="500px")

'C:\\Users\\laoth\\Desktop\\study\\web\\echarts\\RouteBased\\k-cores.html'

## SNN - Shared Near Neighbor

In [25]:
class SSN:
    def __init__(self, snn_routes: Dict[Tuple[str, str], int], method = 'mixed'):
        '''
        Initialize Jarvis-Patrick clustering.
        Parameters:
        - snn_routes: A dictionary of route counts with keys as (source, destination)
        - method: How similarity is calculated ('source', 'destination', or 'mixed').
            'source': similarity calculate by route counts from source airports - only forth
            'destination': similarity calculate by route counts to destination airports - only back
            'mixed': similarity calculate by aggregate route counts - both back and forth
        '''
        if method not in ('source', 'destination', 'mixed'):
            raise ValueError("Invalid method. Choose from 'source', 'destination', 'mixed'.")
        self.method = method
        self.snn_routes = snn_routes.copy()
        self.similarity_df = None
        
    def similarity_matrix_calu(self):
        """Calculate the similarity matrix based on the chosen method."""
        # Extract unique airports
        snn_airports = list(set([airport for route in self.snn_routes.keys() for airport in route]))
        airport_index = {airport: idx for idx, airport in enumerate(snn_airports)}
        
        # Initialize similarity matrix
        similarity_matrix = np.zeros((len(snn_airports), len(snn_airports)), dtype=int)

        # Fill similarity matrix
        for (source, destination), count in self.snn_routes.items():
            i, j = airport_index[source], airport_index[destination]
            if self.method == 'source':
                similarity_matrix[i, j] = count
            elif self.method == 'destination':
                similarity_matrix[j, i] = count
            elif self.method == 'mixed':
                similarity_matrix[i, j] += count
                similarity_matrix[j, i] += count

        # Convert to DataFrame
        self.similarity_df = pd.DataFrame(similarity_matrix, index=snn_airports, columns=snn_airports)

    def non_zero_similarity(self):
        """Generate descriptive statistics for non-zero similarities."""
        if self.similarity_df is None:
            self.similarity_matrix_calu()
        non_zero_counts = (self.similarity_df != 0).sum()
        return non_zero_counts
            
            
    def jp_cluster_calu(self,k=3, T1=2, T2=1):
        """
        Perform Jarvis-Patrick clustering based on SNN similarity.

        Parameters:
        - k: Number of nearest neighbors to consider.
        - T1: Minimum shared neighbors for clustering.
        - T2: Minimum similarity score for clustering.

        Returns:
        - clusters: A dictionary with cluster indices and members.
        """
        if self.similarity_df is None:
            self.similarity_matrix_calu()

        # Find k-nearest neighbors for each airport
        neighbors = {
            airport: self.similarity_df.loc[airport]
            .nlargest(k + 1)  # Include self
            .iloc[1:]         # Exclude self
            .index.tolist()
            for airport in self.similarity_df.index
        }
        self.neighbors = neighbors 
        
        # Build graph based on shared neighbors and similarity thresholds
        G = nx.Graph()
        for airport, neighbor_list in neighbors.items():
            for neighbor in neighbor_list:
                shared_neighbors = len(set(neighbors[airport]) & set(neighbors[neighbor]))
                if shared_neighbors >= T1 and self.similarity_df.loc[airport, neighbor] >= T2:
                    G.add_edge(airport, neighbor)

        # Find connected components (clusters)
        self.clusters = {i: list(c) for i, c in enumerate(nx.connected_components(G))}
        return self.clusters
    
    def create_snn_nodes_count(self,airports_counts_list):
        '''create nodes data for pyechart'''
        try: 
            self.clusters
        except:
            raise ValueError("Run .jp_cluster_calu() first.")
            
        snn_nodes = [airport for cluster in self.clusters.values() for airport in cluster]
        snn_nodes_count = [
            (airport,count)
            for airport,count in airports_counts_list
            if airport in snn_nodes
        ]
        return snn_nodes_count
    
    def create_snn_edges(self):
        '''create edges data for pyechart'''
        try:
            self.clusters
        except:
            raise ValueError("Run .jp_cluster_calu() first.")

        snn_edges = [
            route
            for cluster in self.clusters.values()
            for route in (combinations(cluster, 2))
        ]
        return snn_edges

In [26]:
'''test'''
snn = SSN(snn_routes = routes_dict, method = 'mixed')
JP_clusters = snn.jp_cluster_calu(k=5, T1=2, T2=5)
print(snn.non_zero_similarity().describe())
# print("JP_clusters:",JP_clusters)
# print(snn.create_snn_nodes_count(airports_counts_list))
# print(snn.create_snn_edges())

count    385.000000
mean       4.425974
std        6.940998
min        1.000000
25%        1.000000
50%        2.000000
75%        5.000000
max       79.000000
dtype: float64


### SNN data

In [27]:
snn_s = SSN(snn_routes = routes_dict, method = 'source')
snn_s_clts = snn_s.jp_cluster_calu(k=5, T1=2, T2=1)
snn_s_ngbs = snn_s.neighbors

snn_d = SSN(snn_routes = routes_dict, method = 'destination')
snn_d_clts = snn_d.jp_cluster_calu(k=5, T1=2, T2=1)
snn_d_ngbs = snn_d.neighbors

snn_m = SSN(snn_routes = routes_dict, method = 'mixed')
snn_m_clts = snn_m.jp_cluster_calu(k=5, T1=2, T2=1)
snn_m_ngbs = snn_m.neighbors

snn_nodes_count = {
    'source':snn_s.create_snn_nodes_count(airports_counts_list),
    'destination':snn_d.create_snn_nodes_count(airports_counts_list),
    'mixed':snn_m.create_snn_nodes_count(airports_counts_list)
}
snn_edges = {
    'source':snn_s.create_snn_edges(),
    'destination':snn_d.create_snn_edges(),
    'mixed':snn_m.create_snn_edges()
}

###  Draw SNN

In [28]:
# Initialize the Geo object
c = Geo(init_opts=opts.InitOpts(width="1600px", height="800px")) # Full the screen
c.add_schema(maptype="world")

L_n = len(snn_nodes_count.keys())
L_e = len(snn_edges.keys())

for i, key in enumerate(snn_nodes_count.keys()):
    # Add airport scatter points
    rate = i/L_n
    color = f"rgb({255 - 250*rate},0, 0)"
    c.add(
        f"Airports SNN-{key}",
        data_pair=snn_nodes_count[key],
        type_=ChartType.EFFECT_SCATTER,
        color=color,
        symbol_size=3,
    )
    
for i, key in enumerate(snn_edges.keys()):
    # Add airline routes for different ranges
    rate = i/L_e
    color = f"rgb({255 - 250*rate},160, {250*rate})"
    c.add(
        f"Routes SNN-{key}",
        data_pair=snn_edges[key],  # Replace with your route filtering function
        type_=ChartType.LINES,
        symbol_size=0,
        effect_opts=opts.EffectOpts(
            symbol=SymbolType.ARROW, symbol_size=0, color=color,  # Dynamic RGB color
        ),
        linestyle_opts=opts.LineStyleOpts(
            width=0.2, curve=0, opacity=1, type_="solid",color=color,  # Dynamic RGB color
        ),
    )

c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
c.set_global_opts(title_opts=opts.TitleOpts(title="Aiport and Routes (K-cores)",pos_left = 'center'),
                  legend_opts = opts.LegendOpts(type_ = 'scroll',orient = 'vertical',page_button_position = 'start',
                                               pos_left = 'left'))
c.render('../echarts/RouteBased/snn.html') # save as .html
# c.render_notebook() # if you want to display the graph in notebook, add this line, and set (width="900px", height="500px")

'C:\\Users\\laoth\\Desktop\\study\\web\\echarts\\RouteBased\\snn.html'

## IHCS - Iterated Highly Connected Subgraphs

In [29]:
class IHCS:
    """
    Iterated Highly Connected Subgraphs (IHCS) clustering algorithm implementation.
    Based on: "Hartuv, E., & Shamir, R. (2000). A clustering algorithm based on graph connectivity"
    
    The Iterated HCS version repeatedly applies HCS until convergence.
    """
    
    def __init__(self, min_cluster_size=3):
        """
        Initialize IHCS clustering algorithm.
        
        Args:
            min_cluster_size (int): Minimum number of nodes in a cluster (default: 3)
        """
        self.min_cluster_size = min_cluster_size
        self.labels = {}
        self.clusters = {}
        
    def fit(self, graph):
        """
        Perform IHCS clustering on the input graph.
        
        Args:
            graph (nx.Graph): Input graph to be clustered
            
        Returns:
            dict: Cluster labels for each node
        """
        self.graph = graph.copy()
        
        if self.graph is None or len(self.graph) == 0:
            raise ValueError("No graph provided for clustering")
        
        # Initial clustering
        clustered_graph = self._ihcs_recursive(self.graph)
        
        # Assign labels and group clusters
        self._assign_cluster_labels(clustered_graph)
        self._group_clusters()
        
        return self.labels
    
    def _is_highly_connected(self, graph):
        """
        Check if a graph is highly connected using edge connectivity.
        A graph is highly connected if its edge connectivity is greater than |V|/2.
        """
        n = len(graph)
        if n <= 1:
            return True
        
        try:
            edge_connectivity = nx.edge_connectivity(graph)
            return edge_connectivity > n/2
        except nx.NetworkXError:
            return False
            
    def _get_minimum_cut(self, graph):
        """Get the minimum cut of a graph"""
        try:
            return nx.minimum_edge_cut(graph)
        except nx.NetworkXError:
            return set()
    
    def _remove_edges(self, graph, edges):
        """Remove edges from graph"""
        graph_copy = graph.copy()
        graph_copy.remove_edges_from(edges)
        return graph_copy
    
    def _ihcs_recursive(self, graph):
        """
        Recursive implementation of the Iterated HCS algorithm.
        
        Args:
            graph (nx.Graph: Input graph to be processed
            
        Returns:
            nx.Graph: Processed graph with final clustering
        """
        # Base cases
        if len(graph) < self.min_cluster_size:
            return graph
            
        if not nx.is_connected(graph):
            # Process each component separately
            components = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
            result = nx.Graph()
            for component in components:
                if len(component) >= self.min_cluster_size:
                    result = nx.compose(result, self._ihcs_recursive(component))
                else:
                    result = nx.compose(result, component)
            return result
        
        # Check if current graph is highly connected
        if self._is_highly_connected(graph):
            return graph
            
        # If not highly connected, find minimum cut and split
        min_cut = self._get_minimum_cut(graph)
        if not min_cut:
            return graph
            
        # Split graph and recursively process subgraphs
        split_graph = self._remove_edges(graph, min_cut)
        subgraphs = [split_graph.subgraph(c).copy() for c in nx.connected_components(split_graph)]
        
        # Process each subgraph
        result = nx.Graph()
        for subgraph in subgraphs:
            if len(subgraph) >= self.min_cluster_size:
                processed_subgraph = self._ihcs_recursive(subgraph)
                result = nx.compose(result, processed_subgraph)
            else:
                result = nx.compose(result, subgraph)
                
        return result
    
    def _assign_cluster_labels(self, graph):
        """Assign cluster labels to nodes based on connected components"""
        self.labels = {}
        for cluster_idx, component in enumerate(nx.connected_components(graph), 1):
            if len(component) >= self.min_cluster_size:
                for node in component:
                    self.labels[node] = cluster_idx
            else:
                # Assign small components to cluster 0 (outliers)
                for node in component:
                    self.labels[node] = 0
    
    def _group_clusters(self):
        """
        Group nodes by their cluster labels.
        
        Returns:
            dict: Dictionary where keys are cluster labels and values are lists of nodes
        """
        self.clusters = defaultdict(list)
        for node, label in self.labels.items():
            if label == 0:
                self.clusters['Singletons'].append(node)
            else: 
                self.clusters[f'Subgraph-{label}'].append(node)
        return self.clusters

### IHCS data

In [46]:
# Create IHCS object with minimum cluster size
ihcs = IHCS(min_cluster_size=3)
undirected_community = Community.to_undirected()
ihcs.fit(undirected_community)

# Get clustering results
ihcs_nodes = ihcs.clusters
print("Clusters:", ihcs_nodes)

Clusters: defaultdict(<class 'list'>, {'Singletons': ['ORD', 'BKK', 'HKG', 'NKG', 'TAO', 'HAK', 'CGO', 'KWL', 'DLC', 'SYX', 'HRB', 'TNA', 'FOC', 'SHE', 'NRT', 'ICN', 'TPE', 'SIN', 'LAX', 'SFO', 'JFK', 'LHR', 'ATL', 'SEA', 'YYZ', 'MSP', 'DFW', 'NNG', 'MSY', 'DEN', 'CGQ', 'URC', 'XNN', 'YUL', 'CDG', 'MEX', 'MIA', 'PHX', 'LJG', 'KIX', 'FCO', 'DTW', 'FRA', 'MAD', 'LIS', 'BRU', 'BCN', 'OPO', 'YVR', 'LGA', 'EWR', 'BOS', 'VIE', 'MXP', 'AUH', 'CUN', 'MNL', 'HNL', 'LAS', 'LGW', 'MCO', 'MAN', 'AGP', 'PMI', 'MUC', 'BOM', 'DEL', 'MAA', 'BLR', 'HYD', 'AMS', 'SYD', 'BNE', 'MEL', 'ADL', 'HKT', 'CCU', 'KUL', 'DAC', 'ACE', 'FUE', 'BHX', 'TFS', 'CTA', 'DUS', 'PER', 'DPS', 'CGK', 'AKL', 'DXB', 'BAH', 'DOH', 'PRG', 'EMA', 'CPH', 'TXL', 'ARN', 'COK', 'AYT', 'CGN', 'LHW', 'MTY', 'GDL', 'PUS', 'SAN', 'SLC', 'IAD', 'HFE', 'IAH', 'LCA', 'ATH', 'INC', 'PNQ', 'LPA', 'PHL', 'WNZ', 'NGB', 'IXZ', 'HAM', 'PDX', 'BOG', 'CLO', 'GAU', 'LIM', 'YYC', 'IXB', 'JED', 'KWI', 'HBE', 'SJW', 'OGG', 'DUB', 'ZRH', 'CEB', 'JOG', '

In [31]:
ihcs_nodes_count = create_nodes_count(ihcs_nodes)

In [32]:
ihcs_edges = defaultdict(list)
ihcs_edges['Singletons'] = []
for route in routes_list:
    source, destination = route
    for key, cluster in ihcs_nodes.items():
        if key != 'Singletons':
            if source in cluster and destination in cluster:
                ihcs_edges[key].append(route)
ihcs_edges['Singletons'] += list(set(routes_list)-set([edge for cluster in ihcs_edges.values() for edge in cluster]))

### Draw IHCS

In [33]:
# Initialize the Geo object
c = Geo(init_opts=opts.InitOpts(width="1600px", height="800px")) # Full the screen
c.add_schema(maptype="world")

L_n = len(ihcs_nodes.keys())
L_e = len(ihcs_edges.keys())

for i, key in enumerate(ihcs_nodes.keys()):
    # Add airport scatter points
    rate = i/L_n
    color = f"rgb({255 - 250*rate},0, 0)"
    c.add(
        f"Airports {key}",
        data_pair=ihcs_nodes_count[key],
        type_=ChartType.EFFECT_SCATTER,
        color=color,
        symbol_size=3,
    )

for i, key in enumerate(ihcs_edges.keys()):
    # Add airline routes for different ranges
    rate = i/L_e
    color = f"rgb({255 - 250*rate},160, {250*rate})"
    c.add(
        f"Routes {key}",
        data_pair=ihcs_edges[key],  # Replace with your route filtering function
        type_=ChartType.LINES,
        symbol_size=0,
        effect_opts=opts.EffectOpts(
            symbol=SymbolType.ARROW, symbol_size=3, color=color,  # Dynamic RGB color
        ),
        linestyle_opts=opts.LineStyleOpts(
            width=0.1, curve=0.4, opacity=0.2 + 0.8*rate, type_="solid",color=color,  # Dynamic RGB color
        ),
    )

c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
c.set_global_opts(title_opts=opts.TitleOpts(title="Aiport and Routes (ICHS)",pos_left = 'center'),
                  legend_opts = opts.LegendOpts(type_ = 'scroll',orient = 'vertical',page_button_position = 'start',
                                               pos_left = 'left'))
c.render('../echarts/RouteBased/ihcs.html') # save as .html
# c.render_notebook() # if you want to display the graph in notebook, add this line, and set (width="900px", height="500px")

'C:\\Users\\laoth\\Desktop\\study\\web\\echarts\\RouteBased\\ihcs.html'

## LCMA - Local Clique Merging Algorithm

In [34]:
class LCMA:
    """
    Local Clique Merging Algorithm (LCMA) for analyzing and clustering route networks.
    
    This class implements a graph-based approach to identify and analyze clusters of highly
    connected routes in transportation networks. It uses clique detection and merging
    strategies to identify meaningful route groups while considering edge weights.
    
    The algorithm works in three main steps:
    1. Convert route data into a weighted graph
    2. Identify maximal cliques in the graph
    3. Merge similar cliques based on a similarity threshold
    """
    
    def __init__(self, similarity_threshold: float = 0.5, min_clique_size: int = 3, method = 'undirected'):
        """
        Initialize LCMA router
        
        Args:
            similarity_threshold: Threshold for merging cliques (0.0 to 1.0)
            min_clique_size: Minimum size for considering a clique
            method: Graph type to use for analysis. Options are:
                - 'undirected': Treats routes as bidirectional
                - 'directed': Preserves route directionality
        """
        self.similarity_threshold = similarity_threshold
        self.min_clique_size = min_clique_size
        self.graph = None
        self.cliques = []
        self.clusters = []
        self.method = method
        
    def _create_route_graph(self, routes: Dict[Tuple[str, str], float]) -> Union[nx.Graph, nx.DiGraph]:
        """
        Create a graph from route dictionary
        
        Args:
            routes: Dictionary with (origin, destination) tuples as keys and weights as values
            
        Returns:
        NetworkX graph, either:
        - nx.Graph if self.method == 'undirected'
        - nx.DiGraph if self.method == 'directed'
        """
        if self.method == 'undirected':
            G = nx.Graph()
        if self.method == 'directed':
            G = nx.DiGraph()
        for (origin, dest), weight in routes.items():
            # Add edge with weight. If edge exists, use maximum weight
            if G.has_edge(origin, dest):
                G[origin][dest]['weight'] = max(G[origin][dest]['weight'], weight)
            else:
                G.add_edge(origin, dest, weight=weight)
        return G
    
    def _find_maximal_cliques(self) -> List[Set[str]]:
        """
        Find all maximal cliques in the route graph
        
        Returns:
            List of sets containing nodes in each clique
        """
        if self.method == 'undirected':
            all_cliques = list(nx.find_cliques(self.graph))
        if self.method == 'directed':
            # Convert to undirected for clique finding, as cliques are undefined in directed graphs
            all_cliques = list(nx.find_cliques(self.graph.to_undirected()))
        return [set(c) for c in all_cliques if len(c) >= self.min_clique_size]
    
    def _calculate_clique_similarity(self, clique1: Set[str], clique2: Set[str]) -> float:
        """
        Calculate similarity between two cliques using weighted Jaccard coefficient
        
        Args:
            clique1: First clique
            clique2: Second clique
            
        Similarity score between 0 and 1, where:
            - 0 indicates completely distinct cliques
            - 1 indicates identical cliques with maximum edge weights
        """
        intersection = len(clique1.intersection(clique2))
        union = len(clique1.union(clique2))
        
        # Include edge weights in similarity calculation
        if intersection > 0:
            subgraph1 = self.graph.subgraph(clique1)
            subgraph2 = self.graph.subgraph(clique2)
            avg_weight1 = np.mean([d['weight'] for _, _, d in subgraph1.edges(data=True)])
            avg_weight2 = np.mean([d['weight'] for _, _, d in subgraph2.edges(data=True)])
            weight_factor = (avg_weight1 + avg_weight2) / 2 / 20  # Normalize by max weight
            return (intersection / union) * (1 + weight_factor)
        
        return 0.0
    
    def _merge_similar_cliques(self, cliques: List[Set[str]]) -> List[Set[str]]:
        """
        Merge cliques that have similarity above threshold
        
        Args:
            cliques: List of cliques to merge
            
        Returns:
            List of merged cliques
        """
        merged = True
        while merged:
            merged = False
            for i, j in combinations(range(len(cliques)), 2):
                if i < len(cliques) and j < len(cliques):
                    similarity = self._calculate_clique_similarity(cliques[i], cliques[j])
                    if similarity >= self.similarity_threshold:
                        cliques[i] = cliques[i].union(cliques[j])
                        cliques.pop(j)
                        merged = True
                        break
        return cliques
    
    def _analyze_cluster_connectivity(self, cluster: Set[str]) -> Dict:
        """
        Analyze connectivity patterns within a cluster
        
        Args:
            cluster: Set of nodes in the cluster
            
        Returns:
            Dict containing connectivity metrics
        """
        subgraph = self.graph.subgraph(cluster)
        edges = subgraph.edges(data=True)
        weights = [d['weight'] for _, _, d in edges]
        
        return {
            'size': len(cluster),
            'density': nx.density(subgraph),
            'avg_degree': sum(dict(subgraph.degree()).values()) / len(cluster),
            'avg_weight': np.mean(weights) if weights else 0,
            'max_weight': max(weights) if weights else 0,
            'total_flights': sum(weights) if weights else 0
        }
    
    def find_route_clusters(self, routes: Dict[Tuple[str, str], float]) -> List[Dict]:
        """
        Main method to find and analyze route clusters
        
        Args:
            routes: Dictionary with (origin, destination) tuples as keys and weights as values
            
        Returns:
            List of cluster information dictionaries
        """
        self.graph = self._create_route_graph(routes)
        self.cliques = self._find_maximal_cliques()
        self.clusters = self._merge_similar_cliques(self.cliques)
        
        cluster_info = []
        for i, cluster in enumerate(self.clusters):
            info = {
                'cluster_id': i,
                'nodes': list(cluster),
                'metrics': self._analyze_cluster_connectivity(cluster)
            }
            cluster_info.append(info)
        
        return cluster_info
    
    def get_cluster_routes(self, cluster: Set[str]) -> Dict[Tuple[str, str], float]:
        """
        Get all routes within a cluster
        
        Args:
            cluster: Set of nodes in the cluster
            
        Returns:
            Dictionary of routes with weights
        """
        subgraph = self.graph.subgraph(cluster)
        routes = {}
        for origin, dest, data in subgraph.edges(data=True):
            routes[(origin, dest)] = data['weight']
        return routes

### LCMA data

In [35]:
# Initialize and run LCMA
lcma = LCMA(similarity_threshold=0.2, min_clique_size=3, method = 'directed')
lcma_routes = routes_dict.copy()
clusters = lcma.find_route_clusters(lcma_routes)

In [36]:
# Print results
for cluster in clusters:
    print(f"\nCluster {cluster['cluster_id']}:")
    print(f"Nodes (Airports): {cluster['nodes']}")
    print("Metrics:")
        # Get routes within cluster
    for metric, value in cluster['metrics'].items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.2f}")
        else:
            print(f"  {metric}: {value}")
            
    routes = lcma.get_cluster_routes(set(cluster['nodes']))
    if routes:
        print("Routes (with frequencies):")
        for route, weight in sorted(routes.items()):
            print(route, weight)


Cluster 0:
Nodes (Airports): ['WNZ', 'PVG', 'TNA', 'KUL', 'DPS', 'CEB', 'CGO', 'NNG', 'BKK', 'HKG', 'JOG', 'NRT', 'LAX', 'LJG', 'CAN', 'PER', 'CGQ', 'SYX', 'URC', 'SZX', 'SHA', 'NKG', 'HKT', 'DAC', 'PUS', 'NGO', 'CGK', 'XMN', 'MNL', 'TSN', 'CKG', 'PEK', 'FOC', 'HRB', 'WUH', 'ORD', 'SHE', 'HAK', 'TPE', 'ICN', 'KWE', 'NGB', 'XIY', 'HFE', 'INC', 'HGH', 'SIN', 'XNN', 'TAO', 'KWL', 'DLC', 'CTU', 'CSX', 'TYN', 'KMG', 'KIX']
Metrics:
  size: 56
  density: 0.18
  avg_degree: 19.50
  avg_weight: 6.29
  max_weight: 13
  total_flights: 3435
Routes (with frequencies):
('BKK', 'CAN') 6
('BKK', 'DAC') 6
('BKK', 'HKG') 12
('BKK', 'HKT') 10
('BKK', 'ICN') 10
('BKK', 'KUL') 9
('BKK', 'NRT') 8
('BKK', 'PVG') 5
('BKK', 'SIN') 10
('BKK', 'TPE') 6
('CAN', 'BKK') 6
('CAN', 'CKG') 8
('CAN', 'CTU') 6
('CAN', 'HAK') 5
('CAN', 'HFE') 5
('CAN', 'HGH') 12
('CAN', 'KMG') 5
('CAN', 'NKG') 6
('CAN', 'NRT') 6
('CAN', 'PVG') 6
('CAN', 'SHA') 7
('CAN', 'TAO') 5
('CAN', 'TNA') 5
('CAN', 'TSN') 5
('CAN', 'WNZ') 6
('CAN'

In [37]:
lcma_nodes = {
    cluster['cluster_id'] : cluster['nodes']
    for cluster in clusters
}
lcma_nodes_count = create_nodes_count(lcma_nodes)
lcma_edges = {
    cluster['cluster_id'] : list(lcma.get_cluster_routes(set(cluster['nodes'])).keys())
    for cluster in clusters
}

### Draw IHCS

In [38]:
# Initialize the Geo object
c = Geo(init_opts=opts.InitOpts(width="1600px", height="800px")) # Full the screen
c.add_schema(maptype="world")

L_n = len(lcma_nodes.keys())
L_e = len(lcma_edges.keys())

# Add airport scatter points
for i, key in enumerate(lcma_nodes.keys()):
    rate = i/L_n
    color = f"rgb({255 - 250*rate},0, 0)"
    c.add(
        f"Airports cluster {key}",
        data_pair=lcma_nodes_count[key],
        type_=ChartType.EFFECT_SCATTER,
        color=color,
        symbol_size=3,
    )

for i, key in enumerate(lcma_edges.keys()):
    # Add airline routes for different ranges
    rate = i/L_e
    color = f"rgb({255 - 250*rate},160, {250*rate})"
    c.add(
        f"Routes cluster {key}",
        data_pair=lcma_edges[key],  # Replace with your route filtering function
        type_=ChartType.LINES,
        symbol_size=0,
        effect_opts=opts.EffectOpts(
            symbol=SymbolType.ARROW, symbol_size=3, color=color,  # Dynamic RGB color
        ),
        linestyle_opts=opts.LineStyleOpts(
            width=0.1, curve=0.4, opacity=0.4 + 0.6*rate, type_="solid",color=color,  # Dynamic RGB color
        ),
    )

c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
c.set_global_opts(title_opts=opts.TitleOpts(title="Aiport and Routes (LCMA)",pos_left = 'center'),
                  legend_opts = opts.LegendOpts(type_ = 'scroll',orient = 'vertical',page_button_position = 'start',
                                               pos_left = 'left'))
c.render('../echarts/RouteBased/lcam.html') # save as .html
# c.render_notebook() # if you want to display the graph in notebook, add this line, and set (width="900px", height="500px")

'C:\\Users\\laoth\\Desktop\\study\\web\\echarts\\RouteBased\\lcam.html'

## Divisive methods

In [39]:
import networkx as nx
from networkx.algorithms.community import girvan_newman
import matplotlib.pyplot as plt

# Create an example graph
G = nx.DiGraph()

# Add nodes and edges to form two distinct communities
edges_community_1 = [(1, 2), (2, 3), (3, 4), (4, 1)]  # First community (Cycle)
edges_community_2 = [(5, 6), (6, 7), (7, 8), (8, 5)]  # Second community (Cycle)
edges_between = [(4, 5)]  # Bridge edge connecting the two communities

# Add all edges to the graph
G.add_edges_from(edges_community_1 + edges_community_2 + edges_between)

import itertools
iteration = 4
comp = girvan_newman(G)
for communities in itertools.islice(comp, iteration):
    print(tuple(sorted(c) for c in communities))

([1, 2, 3, 4], [5, 6, 7, 8])
([1, 4], [2, 3], [5, 6, 7, 8])
([1, 4], [2, 3], [5, 8], [6, 7])
([1], [2, 3], [4], [5, 8], [6, 7])


In [40]:
import itertools
k = 3
comp = girvan_newman(G)
limited = itertools.takewhile(lambda c: len(c) <= k, comp)
for communities in limited:
    print(tuple(sorted(c) for c in communities))

([1, 2, 3, 4], [5, 6, 7, 8])
([1, 4], [2, 3], [5, 6, 7, 8])


In [41]:
G = nx.DiGraph()
edges = {
    (1, 2):1,
    (2, 3):2,
    (3, 4):3,
    (4, 1):4,
    (1, 4):3,
}
weighted_edges = [(u, v, weight) for (u, v), weight in edges.items()]
G.add_weighted_edges_from(weighted_edges)
G.edges(data=True)

OutEdgeDataView([(1, 2, {'weight': 1}), (1, 4, {'weight': 3}), (2, 3, {'weight': 2}), (3, 4, {'weight': 3}), (4, 1, {'weight': 4})])

In [42]:
G = nx.DiGraph()
edges = [
    (1, 2, 1),
    (2, 3, 2),
    (3, 4, 3),
    (4, 1, 4),
    (1,4,5),
     ]
G.add_weighted_edges_from(edges)
G.edges(data='weight')

OutEdgeDataView([(1, 2, 1), (1, 4, 5), (2, 3, 2), (3, 4, 3), (4, 1, 4)])

In [43]:
from networkx import edge_betweenness_centrality as betweenness
def most_central_edge(G):
    centrality = betweenness(G, weight="weight")
    return max(centrality, key=centrality.get)

comp = girvan_newman(G, most_valuable_edge=most_central_edge)
tuple(sorted(c) for c in next(comp))


([1, 4], [2, 3])

In [44]:
G.edges(data='weight')

OutEdgeDataView([(1, 2, 1), (1, 4, 5), (2, 3, 2), (3, 4, 3), (4, 1, 4)])